In [1]:
with open('paloozabot.log','r') as file:
    log_data = file.read()

In [5]:
from utils.listify_logs import listify_log
from utils.jsonify_log import jsonify_log
log_list = listify_log(log_data)
jsonify_log(log_list[0]) # test

{'Datetime': '2024-05-01T15:33:36',
 'Level': 'WARNING',
 'Module': 'discord.client',
 'Context': 'client:295:__init__',
 'Message': 'PyNaCl is not installed, voice will NOT be supported'}

In [ ]:
# start with a log then get a json
# we turn it into a single tabular record then we transfer it to the database
# the AI is giving us different jsons (3 different kinds)
# it spits out a json and we need to know what table it belongs to.
# we'll use the length of the dict for this rather than asking gpt to figure it out